# START

In [1]:
import sys
sys.path.append(r'E:\python-test\study-python')

## 数据集

In [2]:
from datasets import load_dataset
loaded_dataset = load_dataset(r"E:\python-test\DATASETS\zxbsmkwebnovel_cn")

In [3]:
loaded_dataset['train'][0]

{'input': '有其主必有其仆，咱们张大少爷的话还没有说完，张石头的口水就已经流出了嘴角，张大少爷又趁热打铁，鼓动道：“你想想，熊瑚那个丫鬟秀儿长得有多漂亮啊？你如果一辈子窝在这个临清城里，有没有可能讨到这么漂亮的媳妇？这么好的机会放在面前，难道你想错过，以后就只抱着一个黄脸婆过下半辈子？”\n张大少爷的话句句都打中要害，二十出头还没讨到老婆的张石头难免动心无比，不过仔细考虑了一下后，张石头还是哭丧着脸说道：“少爷，不是小的拦着你，只是这事可不是开玩笑，你从小就没离开过临清城，忽然之间悄悄离开家进京，老爷和十一位夫人怎么可能放心得下？只怕你前脚出了城，老爷后脚就派人把你抓回家了。而且还有一点，那只母老虎熊瑚实在太凶了，我们进了京城就算能找得到她，只怕少爷你也没办法把她娶回家啊？”\n“怕什么？只要功夫深，铁棒磨成针，咱们只要再见到熊瑚那只母老虎，就肯定有办法把她连人带丫鬟一起弄回临清。”早有打算却不能明言的张大少爷随口瞎扯，又皱眉道：“至于老头子这边，是得防着他派人来抓我们，否则他回来发现少爷我不在家，肯定要到处去找。”琢磨到这里，张大少爷一拍大腿，喜道：“对了，有办法让老头子抓不到咱们了。”\n“什么办法？”张石头赶紧问道。张大少爷奸笑道：“别急，到时候你就知道了。别罗嗦了，快回家收拾行李，乘着还没天黑赶快动身。”',
 'instruction': '给你一段话，帮我继续写下去',
 'output': '张石头将信将疑，又实在细胳臂扭不过大腿，只得咬牙决定赌上一把，“好吧，既然少爷你执意要去，小的就陪你走上这一趟。不过将来老爷如果追究起来，少爷你可要给小的说几句好话。”张大少爷咧嘴一笑，安慰道：“放心吧，如果老头子真的把我们抓住了，少爷我全抗下来，不关你事。”\n……\n好不容易说服张石头同行，当下张大少爷和张石头一主一仆两个活宝立即行动起来，先是翻墙回家分头行动，张石头摸到张大少爷房间里收拾衣服行李，张大少爷则通过记忆找出前任张大少爷偷配的银库钥匙摸进张老财的书房，打开张老财用来收藏贵重财物的密库，搜刮出厚厚一大叠银票和满满一大包金叶子金元宝，又装了不少极其贵重的珠宝，这才偷偷摸回自己的房间与张石头会合。\n刚进门，张石头就举起一个包裹说岛：“少爷，东西都准备好了，按你的吩咐，你和我都只带了一套换洗的衣服，还有一点零用的碎银子。”张大少爷点点头，又让张

### TOKENIZER

In [7]:
from transformers import AutoTokenizer
text_data = loaded_dataset['train'][0]['input']
# 创建 tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2", cache_dir=r'./TOKENIZER')  # 可以根据需要选择基础模型


In [9]:
# 加载数据集
dataset = loaded_dataset

# 应用tokenizer到数据集
def apply_tokenizer(example):
    return tokenizer(example['input'], example['instruction'], example['output'], truncation=True)

tokenized_dataset = dataset.map(apply_tokenizer, batched=True)

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [17]:
tokenized_dataset['train'][0].keys()

dict_keys(['input', 'instruction', 'output', 'input_ids', 'attention_mask', 'labels'])

In [19]:
tokenized_dataset.set_format("torch", columns=["input_ids",'attention_mask', "labels"])

## infini-llama

In [20]:
%load_ext autoreload
%autoreload 2

from tot.infini_llama import *
from transformers import LlamaConfig

In [21]:
from transformers import LlamaConfig, LlamaForCausalLM, Trainer, TrainingArguments

config = LlamaConfig()
config.hidden_size=256
config.intermediate_size=172
config.max_position_embeddings=256
config.num_attention_heads=1
config.num_hidden_layers=1
config.num_key_value_heads=1
config.rope_theta=10.0
config.vocab_size=200

# 配置模型
model = LlamaForCausalLM(config=config)


In [24]:
# 3. 设置训练参数
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=1,
    weight_decay=0.01,
    logging_dir="./logs",
)


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:

# 4. 进行训练
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()